In [1]:
#Cargar librerías
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt

In [2]:
#Cargar el dataset
df = pd.read_csv("dirty_cafe_sales.csv")

### Análisis inicial y limpieza 

In [3]:
print(df.head())

  Transaction ID    Item Quantity Price Per Unit Total Spent  Payment Method  \
0    TXN_1961373  Coffee        2            2.0         4.0     Credit Card   
1    TXN_4977031    Cake        4            3.0        12.0            Cash   
2    TXN_4271903  Cookie        4            1.0       ERROR     Credit Card   
3    TXN_7034554   Salad        2            5.0        10.0         UNKNOWN   
4    TXN_3160411  Coffee        2            2.0         4.0  Digital Wallet   

   Location Transaction Date  
0  Takeaway       2023-09-08  
1  In-store       2023-05-16  
2  In-store       2023-07-19  
3   UNKNOWN       2023-04-27  
4  In-store       2023-06-11  


In [4]:
print(df.shape)

(10000, 8)


In [5]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              9667 non-null   object
 2   Quantity          9862 non-null   object
 3   Price Per Unit    9821 non-null   object
 4   Total Spent       9827 non-null   object
 5   Payment Method    7421 non-null   object
 6   Location          6735 non-null   object
 7   Transaction Date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB
None


In [6]:
df.nunique()

Transaction ID      10000
Item                   10
Quantity                7
Price Per Unit          8
Total Spent            19
Payment Method          5
Location                4
Transaction Date      367
dtype: int64

In [7]:
df['Quantity'] = pd.to_numeric(df['Quantity'], errors='coerce')
df['Price Per Unit'] = pd.to_numeric(df['Price Per Unit'], errors='coerce')
df['Total Spent'] = pd.to_numeric(df['Total Spent'], errors='coerce')


In [8]:
print(df.describe())

          Quantity  Price Per Unit  Total Spent
count  9521.000000     9467.000000  9498.000000
mean      3.028463        2.949984     8.924352
std       1.419007        1.278450     6.009919
min       1.000000        1.000000     1.000000
25%       2.000000        2.000000     4.000000
50%       3.000000        3.000000     8.000000
75%       4.000000        4.000000    12.000000
max       5.000000        5.000000    25.000000


In [9]:
df['Item'].unique()

array(['Coffee', 'Cake', 'Cookie', 'Salad', 'Smoothie', 'UNKNOWN',
       'Sandwich', nan, 'ERROR', 'Juice', 'Tea'], dtype=object)

In [10]:
df['Price Per Unit'].unique()

array([2. , 3. , 1. , 5. , 4. , 1.5, nan])

In [11]:
df['Item'] = df['Item'].str.strip().str.capitalize()

In [12]:
# Ver valores únicos posibles de "Price Per Unit" para cada valor en "Item"
check_item = ['Coffee', 'Cake', 'Cookie', 'Salad', 'Smoothie', 'Unknown',
              'Sandwich', 'Error', 'Juice', 'Tea']

for item in check_item:
    values = df[df['Item'] == item]['Price Per Unit'].dropna().unique()
    print(f'Item: {item} - Valores únicos en "Price Per Unit": {sorted(values)}')

Item: Coffee - Valores únicos en "Price Per Unit": [2.0]
Item: Cake - Valores únicos en "Price Per Unit": [3.0]
Item: Cookie - Valores únicos en "Price Per Unit": [1.0]
Item: Salad - Valores únicos en "Price Per Unit": [5.0]
Item: Smoothie - Valores únicos en "Price Per Unit": [4.0]
Item: Unknown - Valores únicos en "Price Per Unit": [1.0, 1.5, 2.0, 3.0, 4.0, 5.0]
Item: Sandwich - Valores únicos en "Price Per Unit": [4.0]
Item: Error - Valores únicos en "Price Per Unit": [1.0, 1.5, 2.0, 3.0, 4.0, 5.0]
Item: Juice - Valores únicos en "Price Per Unit": [3.0]
Item: Tea - Valores únicos en "Price Per Unit": [1.5]


In [13]:
#Verificar cantidad de valores nulos y no nulos para cada valor de Item
check_item = ['Coffee', 'Cake', 'Cookie', 'Salad', 'Smoothie', 'Unknown',
              'Sandwich', 'Error', 'Juice', 'Tea']

for item in check_item:
    subset = df[df['Item'] == item]['Price Per Unit']
    count_non_null = subset.notna().sum()
    count_null = subset.isna().sum()
    print(f'Item: {item} - Valores no nulos en "Price Per Unit": {count_non_null}, nulos: {count_null}')

Item: Coffee - Valores no nulos en "Price Per Unit": 1108, nulos: 57
Item: Cake - Valores no nulos en "Price Per Unit": 1085, nulos: 54
Item: Cookie - Valores no nulos en "Price Per Unit": 1026, nulos: 66
Item: Salad - Valores no nulos en "Price Per Unit": 1082, nulos: 66
Item: Smoothie - Valores no nulos en "Price Per Unit": 1036, nulos: 60
Item: Unknown - Valores no nulos en "Price Per Unit": 326, nulos: 18
Item: Sandwich - Valores no nulos en "Price Per Unit": 1082, nulos: 49
Item: Error - Valores no nulos en "Price Per Unit": 279, nulos: 13
Item: Juice - Valores no nulos en "Price Per Unit": 1110, nulos: 61
Item: Tea - Valores no nulos en "Price Per Unit": 1023, nulos: 66


In [14]:
#Completar valores nulos con el Unit Price de cada Item
condition_coffee = (df['Item'] == 'Coffee') & (df['Price Per Unit'].isna())
df.loc[condition_coffee, 'Price Per Unit'] = 2

condition_cake = (df['Item'] == 'Cake') & (df['Price Per Unit'].isna())
df.loc[condition_cake, 'Price Per Unit'] = 3

condition_cookie= (df['Item'] == 'Cookie') & (df['Price Per Unit'].isna())
df.loc[condition_cookie, 'Price Per Unit'] = 1

condition_salad = (df['Item'] == 'Salad') & (df['Price Per Unit'].isna())
df.loc[condition_salad, 'Price Per Unit'] = 5

condition_smoothie = (df['Item'] == 'Smoothie') & (df['Price Per Unit'].isna())
df.loc[condition_smoothie, 'Price Per Unit'] = 4

condition_sandwich = (df['Item'] == 'Sandwich') & (df['Price Per Unit'].isna())
df.loc[condition_sandwich, 'Price Per Unit'] = 4

condition_juice = (df['Item']== 'Juice') & (df['Price Per Unit'].isna())
df.loc[condition_juice,'Price Per Unit'] = 3

condition_tea = (df["Item"]== 'Tea') & (df["Price Per Unit"].isna())
df.loc[condition_tea, 'Price Per Unit'] = 1.5

In [15]:
check_item = ['Coffee', 'Cake', 'Cookie', 'Salad', 'Smoothie', 'Unknown',
              'Sandwich', 'Error', 'Juice', 'Tea']

for item in check_item:
    subset = df[df['Item'] == item]['Price Per Unit']
    count_non_null = subset.notna().sum()
    count_null = subset.isna().sum()
    print(f'Item: {item} - Valores no nulos en "Price Per Unit": {count_non_null}, nulos: {count_null}')

Item: Coffee - Valores no nulos en "Price Per Unit": 1165, nulos: 0
Item: Cake - Valores no nulos en "Price Per Unit": 1139, nulos: 0
Item: Cookie - Valores no nulos en "Price Per Unit": 1092, nulos: 0
Item: Salad - Valores no nulos en "Price Per Unit": 1148, nulos: 0
Item: Smoothie - Valores no nulos en "Price Per Unit": 1096, nulos: 0
Item: Unknown - Valores no nulos en "Price Per Unit": 326, nulos: 18
Item: Sandwich - Valores no nulos en "Price Per Unit": 1131, nulos: 0
Item: Error - Valores no nulos en "Price Per Unit": 279, nulos: 13
Item: Juice - Valores no nulos en "Price Per Unit": 1171, nulos: 0
Item: Tea - Valores no nulos en "Price Per Unit": 1089, nulos: 0
